In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Check if files exist
model_path = 'model.h5'
gender_encoder_path = 'lable_encoder_gender.pkl'
geo_encoder_path = 'ohe_geo.pkl'
scaler_path = 'scaler.pkl'

In [2]:
model = load_model(model_path)
    
with open(gender_encoder_path, 'rb') as file:
    label_encoder_gender = pickle.load(file)
        
with open(geo_encoder_path, 'rb') as file:
    ohe_geo = pickle.load(file)
        
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)
        
   

In [3]:
sample_input = {
    'CreditScore':619,
    'Geography':'France',
    'Gender':'Female',
    'Age':42,
    'Tenure':2,
    'Balance':0,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':101348.88
}

In [4]:
#ohe


In [5]:
geo_encoded = ohe_geo.transform([[sample_input['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\singh\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df=pd.DataFrame([sample_input])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0,1,1,1,101348.88


In [7]:
#encode categorical variable


In [8]:
input_df['Gender']=label_encoder_gender.transform([sample_input['Gender']])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0,1,1,1,101348.88


In [9]:
input_df=pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0,1,1,1,101348.88,1.0,0.0,0.0


In [10]:
#scalling the input dataa

In [11]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [12]:
#prediction

In [13]:
prediction=model.predict(input_scaled)
prediction


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step


array([[0.13806678]], dtype=float32)

In [14]:
prediction_proba=prediction[0][0]

In [15]:
prediction_proba

0.13806678

In [16]:
if prediction_proba>0.5:
    print("customer likely to churn")
else:
    print("customer not likely to churn")

customer not likely to churn
